In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
df = pd.read_csv('data/wine_data.csv')
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

input_size = 13
output_size = 3
hidden_size = 100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [4]:
model = Net()
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [5]:
checkpoint = torch.load('models/wine_checkpoint.pth')

In [6]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [8]:
start_epoch = checkpoint['epoch']
start_epoch

99

In [9]:
loss = checkpoint['loss']
loss

tensor(0.2540, requires_grad=True)

In [10]:
model.train()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [11]:
EPOCHS = 500
for epoch in range(start_epoch, EPOCHS):
    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

Epoch: 100, Loss: 0.2454
Epoch: 110, Loss: 0.1580
Epoch: 120, Loss: 0.1198
Epoch: 130, Loss: 0.0942
Epoch: 140, Loss: 0.0790
Epoch: 150, Loss: 0.0686
Epoch: 160, Loss: 0.0612
Epoch: 170, Loss: 0.0559
Epoch: 180, Loss: 0.0520
Epoch: 190, Loss: 0.0489
Epoch: 200, Loss: 0.0465
Epoch: 210, Loss: 0.0445
Epoch: 220, Loss: 0.0428
Epoch: 230, Loss: 0.0413
Epoch: 240, Loss: 0.0400
Epoch: 250, Loss: 0.0388
Epoch: 260, Loss: 0.0376
Epoch: 270, Loss: 0.0366
Epoch: 280, Loss: 0.0355
Epoch: 290, Loss: 0.0344
Epoch: 300, Loss: 0.0333
Epoch: 310, Loss: 0.0320
Epoch: 320, Loss: 0.0304
Epoch: 330, Loss: 0.0286
Epoch: 340, Loss: 0.0267
Epoch: 350, Loss: 0.0248
Epoch: 360, Loss: 0.9531
Epoch: 370, Loss: 0.5064
Epoch: 380, Loss: 0.5791
Epoch: 390, Loss: 0.4422
Epoch: 400, Loss: 0.3948
Epoch: 410, Loss: 0.3272
Epoch: 420, Loss: 0.2349
Epoch: 430, Loss: 0.1852
Epoch: 440, Loss: 0.1500
Epoch: 450, Loss: 0.1173
Epoch: 460, Loss: 0.0905
Epoch: 470, Loss: 0.0691
Epoch: 480, Loss: 0.0596
Epoch: 490, Loss: 0.0557


In [12]:
model.eval()
predict_out = model(Xtest_)
_, predict_y = torch.max(predict_out, 1)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy: {accuracy_score(ytest_, predict_y):.4f}')
print(f'Precision: {precision_score(ytest_, predict_y, average="micro"):.4f}')
print(f'Recall: {recall_score(ytest_, predict_y, average="micro"):.4f}')
print(f'F1 Score: {f1_score(ytest_, predict_y, average="micro"):.4f}')

Accuracy: 0.8889
Precision: 0.8889
Recall: 0.8889
F1 Score: 0.8889
